In [1]:
import sys

sys.path.append("/gstore/project/paneldesign/pkg")

sys.path.append("/gstore/home/taol9/gitsss/yo/tao")
# import primer3
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import Entrez
import csv
import pandas as pd
from collections import defaultdict,OrderedDict
from collections import Counter
import matplotlib.pyplot as plt

import pandas as pd
import os
import pickle

#check GC

#https://biopython.org/docs/1.75/api/Bio.SeqUtils.MeltingTemp.html
from Bio.SeqUtils import MeltingTemp as mt
from Bio.Seq import Seq

In [2]:
# !pip install --target /gstore/project/paneldesign/pkg dill

In [2]:
# pip install --target /gstore/project/paneldesign/pkg biopython

# flex probe Design

When designing custom probes, consider the
following:
• GC content should be between 44 − 72% for each
25 bp probe half.
• Avoid homopolymer repeats.
• Avoid overlap with annotated repeat or low complexity
sequences.
• If possible, design probes for coding regions of
mRNA as opposed to untranslated regions.
# The 25th nucleotide of the probe (3' most nucleotide of the LHS probe) must be a T. The complementary nucleotide in the target RNAmust be an A.
• Avoid common single nucleotide polymorphisms
(SNPs) and potential mismatches at the ligation
junction. Refer to the UCSC Genome Browser and
the Single Nucleotide Polymorphism Database
(dbSNP). If avoiding SNPs is not possible, SNPs and
mismatches should be at least four bp away from
the ligation junction.
• If probes can bind to sequences other than the
target mRNA sequence, off-target signal may be
observed. To check for off-target homology, align
the probe sequence to the reference transcriptome
using the Basic Local Alignment Search Tool
(BLAST). Matches to off-target genes should have
at least five mismatches in at least one of the LHS
or RHS probes to prevent efficient hybridization.
• Designing three probe pairs per target mRNA
is recommended, especially for low expressing
genes. However, if the gene is not long enough or
there aren't enough specific 50 bp regions, fewer
than three probe pairs is acceptable.
## Probe pairs should not overlap to avoid competition
between probes for the same binding site in
the target RNA.
• Add new probe sequences to the probe set reference
CSV file. Refer to the Analysis section for
more information.

# structure

left_arm="CCTTGGCACCCGAGAATTCCA"+target_LHS
right_arm="/5Phos/"+target_RHS+"ACGCGGTTAGCACGTA-NN"+probeset_barcode_8nt+"CGGTCCTAGCAA"

fulllib:
p7+i7+left_arm+right_arm+10xumi+10xcellbarcode+read1seqprimer+i5+p5

left_arm_snv=snv_dialout_adaptor+"CCTTGGCACCCGAGAATTCCA"+target_LHS
padlock=right_arm+left_arm_snv

# check strand,make sure statred with ATG
mRNA is the template strand, starting from ATG....
# probe strand is cs of mRNA strand

In [21]:
# mutation codes are 1 based index, python is 0 based, convert that!
# for rna probes, use shorter arm

# add one bp gap for the mutation position
def get_target_LHS(template,mutation,gap=1):
    mutation_position=int(mutation[2:-3])-1
    ref_base=mutation[-3]
    mutation_base=mutation[-1]
    if template[mutation_position]==ref_base:
        for l in range(25+gap,28+gap,1):
#         for l in range(22,23,1):

            mystring=Seq(template[mutation_position:mutation_position+l])
            # print (mystring)
            gc=mt.Tm_GC(mystring)
            if 44<gc<70:
                return (str(mystring.reverse_complement()), 
                       str(Seq(mutation_base+str(mystring[1:])).reverse_complement()))
                  
                break
    else:
        return "wrong input", mutation_position, mutation_base, ref_base,mutation

In [20]:
#add extra gap for the size of gap - 1

def get_target_RHS(template,mutation,gap=2):
    mutation_position=int(mutation[2:-3])-1
    ref_base=mutation[-3]
    mutation_base=mutation[-1]
    if template[mutation_position]==ref_base:
        for l in range(25+gap,28+gap,1):
#         for l in range(22,23,1):
            mystring=Seq(template[mutation_position-l:mutation_position])
            gc=mt.Tm_GC(mystring)
            if 44<gc<70:
                return str(mystring.reverse_complement())
                break
    else:
        return "wrong input", mutation_base, ref_base, mutation

# junction frequency

Counter({'AA': 135,
         'AC': 119,
         'AG': 184,
         'AT': 5916,
         'CA': 11022,
         'CC': 129,
         'CG': 169,
         'CT': 8252,
         'GA': 146,
         'GC': 109,
         'GG': 157,
         'GT': 132,
         'TA': 4313,
         'TC': 7254,
         'TG': 9497,
         'TT': 6423})

In [11]:
from collections import Counter

In [16]:
(Counter(sorted(x)))


Counter({'AA': 135,
         'AC': 119,
         'AG': 184,
         'AT': 5916,
         'CA': 11022,
         'CC': 129,
         'CG': 169,
         'CT': 8252,
         'GA': 146,
         'GC': 109,
         'GG': 157,
         'GT': 132,
         'TA': 4313,
         'TC': 7254,
         'TG': 9497,
         'TT': 6423})

## define adaptor


In [4]:
barcodes=pd.read_csv('probe-barcodes-fixed-rna-profiling.txt',sep='\t', header=None)
barcodes=barcodes[[1,2]].drop_duplicates()
for i,r in barcodes.iterrows():
    print (r[2],'=', '"'+r[1]+'"')

BC001 = "ACTTTAGG"
BC002 = "AACGGGAA"
BC003 = "AGTAGGCT"
BC004 = "ATGTTGAC"
BC005 = "ACAGACCT"
BC006 = "ATCCCAAC"
BC007 = "AAGTAGAG"
BC008 = "AGCTGTGA"
BC009 = "ACAGTCTG"
BC010 = "AGTGAGTG"
BC011 = "AGAGGCAA"
BC012 = "ACTACTCA"
BC013 = "ATACGTCA"
BC014 = "ATCATGTG"
BC015 = "AACGCCGA"
BC016 = "ATTCGGTT"


In [5]:
BC001 = "ACTTTAGG"
BC002 = "AACGGGAA"
BC003 = "AGTAGGCT"
BC004 = "ATGTTGAC"
BC005 = "ACAGACCT"
BC006 = "ATCCCAAC"
BC007 = "AAGTAGAG"
BC008 = "AGCTGTGA"
BC009 = "ACAGTCTG"
BC010 = "AGTGAGTG"
BC011 = "AGAGGCAA"
BC012 = "ACTACTCA"
BC013 = "ATACGTCA"
BC014 = "ATCATGTG"
BC015 = "AACGCCGA"
BC016 = "ATTCGGTT"

In [22]:
# use B013
# use LNA base primer for snv_dialout when run with rna probes
# max lenght set to 120 for rna probes
BC013 = "ATACGTCA"
snv_dialout_adaptor=Mly1_F_w='GCAACGACTCCACACTCATA'

def flex_padlock_gap(template,mutation,probeset_barcode_8nt="ATACGTCA",snv_dialout_adaptor='GCAACGACTCCACACTCATA'):
    target_LHS=get_target_LHS(template,mutation)

    target_RHS=get_target_RHS(template,mutation)
    # left_arm0=snv_dialout_adaptor+"CCTTGGCACCCGAGAATTCCA"+target_LHS[0]
    # left_arm1=snv_dialout_adaptor+"CCTTGGCACCCGAGAATTCCA"+target_LHS[1]
    left_arm=snv_dialout_adaptor+"CCTTGGCACCCGAGAATTCCA"+target_LHS[0][:-1]
    # left_arm1=snv_dialout_adaptor+"CCTTGGCACCCGAGAATTCCA"+target_LHS[1]
    right_arm="/5Phos/"+target_RHS[2:]+"ACGCGGTTAGCACGTANN"+probeset_barcode_8nt+"CGGTCCTAGCAA"
    #fulllib:
    #p7+i7+left_arm+right_arm+10xumi+10xcellbarcode+read1seqprimer+i5+p5
#     left_arm_snv=snv_dialout_adaptor+"CCTTGGCACCCGAGAATTCCA"+target_LHS
    # padlock_ref=right_arm+snv_dialout_adaptor+"CCTTGGCACCCGAGAATTCCA"+target_LHS[0]
    # padlock_alt=right_arm+snv_dialout_adaptor+"CCTTGGCACCCGAGAATTCCA"+target_LHS[1]
    padlock=right_arm+snv_dialout_adaptor+"CCTTGGCACCCGAGAATTCCA"+target_LHS[0][:-1]

#     print (len(target_RHS+"ACGCGGTTAGCACGTANN"+probeset_barcode_8nt+"CGGTCCTAGCAA")+len(left_arm0), len(target_LHS[0]),len(target_RHS))

    return {'LHS':left_arm, 'target_LHS_ref':target_LHS[0], 'target_LHS_mut':target_LHS[1],'target_RHS':target_RHS,"RHS":right_arm, "padlock":padlock}

In [10]:
# convert to padlock probe

In [23]:
KRAS_CDS='''ATGACTGAATATAAACTTGTGGTAGTTGGAGCTGGTGGCGTAGGCAAGAGTGCCTTGACGATACAGCTAATTCAGAATCATTTTGTGGACGAATATGATCCAACAATAGAGGATTCCTAC
AGGAAGCAAGTAGTAATTGATGGAGAAACCTGTCTCTTGGATATTCTCGACACAGCAGGT
CAAGAGGAGTACAGTGCAATGAGGGACCAGTACATGAGGACTGGGGAGGGCTTTCTTTGT
GTATTTGCCATAAATAATACTAAATCATTTGAAGATATTCACCATTATAGAGAACAAATT
AAAAGAGTTAAGGACTCTGAAGATGTACCTATGGTCCTAGTAGGAAATAAATGTGATTTG
CCTTCTAGAACAGTAGACACAAAACAGGCTCAGGACTTAGCAAGAAGTTATGGAATTCCT
TTTATTGAAACATCAGCAAAGACAAGACAGAGAGTGGAGGATGCTTTTTATACATTGGTG
AGAGAGATCCGACAATACAGATTGAAAAAAATCAGCAAAGAAGAAAAGACTCCTGGCTGT
GTGAAAATTAAAAAATGCATTATAATGTAA'''
KRAS_CDS=KRAS_CDS.replace('\n','')



In [23]:
KRAS_CDS[34]

'G'

In [24]:
KRAS_flex=flex_padlock_gap(KRAS_CDS,'c.34G>T')


In [25]:
KRAS_flex

{'LHS_mut': 'GCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCATCAAGGCACTCTTGCCTACGCCACA',
 'LHS_ref': 'GCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCATCAAGGCACTCTTGCCTACGCCACC',
 'RHS': '/5Phos/AGCTCCAACTACCACAAGTTTATATACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAA',
 'padlock_alt': '/5Phos/AGCTCCAACTACCACAAGTTTATATACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCATCAAGGCACTCTTGCCTACGCCACA',
 'padlock_ref': '/5Phos/AGCTCCAACTACCACAAGTTTATATACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCATCAAGGCACTCTTGCCTACGCCACC',
 'target_LHS_mut': 'TCAAGGCACTCTTGCCTACGCCACA',
 'target_LHS_ref': 'TCAAGGCACTCTTGCCTACGCCACC',
 'target_RHS': 'AGCTCCAACTACCACAAGTTTATAT'}

In [25]:
KRAS_flex_gap=flex_padlock_gap(KRAS_CDS,'c.34G>T')
KRAS_flex_gap

{'LHS': 'GCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCAGTCAAGGCACTCTTGCCTACGCCAC',
 'target_LHS_ref': 'GTCAAGGCACTCTTGCCTACGCCACC',
 'target_LHS_mut': 'GTCAAGGCACTCTTGCCTACGCCACA',
 'target_RHS': 'AGCTCCAACTACCACAAGTTTATATTC',
 'RHS': '/5Phos/CTCCAACTACCACAAGTTTATATTCACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAA',
 'padlock': '/5Phos/CTCCAACTACCACAAGTTTATATTCACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCAGTCAAGGCACTCTTGCCTACGCCAC'}

In [26]:
TP63_CDS='''ATGAATTTTGAAACTTCACGGTGTGCCACCCTACAGTACTGCCCTGACCCTTACATCCAGCGTTTCGTAG
AAACCCCAGCTCATTTCTCTTGGAAAGAAAGTTATTACCGATCCACCATGTCCCAGAGCACACAGACAAA
TGAATTCCTCAGTCCAGAGGTTTTCCAGCATATCTGGGATTTTCTGGAACAGCCTATATGTTCAGTTCAG
CCCATTGACTTGAACTTTGTGGATGAACCATCAGAAGATGGTGCGACAAACAAGATTGAGATTAGCATGG
ACTGTATCCGCATGCAGGACTCGGACCTGAGTGACCCCATGTGGCCACAGTACACGAACCTGGGGCTCCT
GAACAGCATGGACCAGCAGATTCAGAACGGCTCCTCGTCCACCAGTCCCTATAACACAGACCACGCGCAG
AACAGCGTCACGGCGCCCTCGCCCTACGCACAGCCCAGCTCCACCTTCGATGCTCTCTCTCCATCACCCG
CCATCCCCTCCAACACCGACTACCCAGGCCCGCACAGTTTCGACGTGTCCTTCCAGCAGTCGAGCACCGC
CAAGTCGGCCACCTGGACGTATTCCACTGAACTGAAGAAACTCTACTGCCAAATTGCAAAGACATGCCCC
ATCCAGATCAAGGTGATGACCCCACCTCCTCAGGGAGCTGTTATCCGCGCCATGCCTGTCTACAAAAAAG
CTGAGCACGTCACGGAGGTGGTGAAGCGGTGCCCCAACCATGAGCTGAGCCGTGAATTCAACGAGGGACA
GATTGCCCCTCCTAGTCATTTGATTCGAGTAGAGGGGAACAGCCATGCCCAGTATGTAGAAGATCCCATC
ACAGGAAGACAGAGTGTGCTGGTACCTTATGAGCCACCCCAGGTTGGCACTGAATTCACGACAGTCTTGT
ACAATTTCATGTGTAACAGCAGTTGTGTTGGAGGGATGAACCGCCGTCCAATTTTAATCATTGTTACTCT
GGAAACCAGAGATGGGCAAGTCCTGGGCCGACGCTGCTTTGAGGCCCGGATCTGTGCTTGCCCAGGAAGA
GACAGGAAGGCGGATGAAGATAGCATCAGAAAGCAGCAAGTTTCGGACAGTACAAAGAACGGTGATGGTA
CGAAGCGCCCGTTTCGTCAGAACACACATGGTATCCAGATGACATCCATCAAGAAACGAAGATCCCCAGA
TGATGAACTGTTATACTTACCAGTGAGGGGCCGTGAGACTTATGAAATGCTGTTGAAGATCAAAGAGTCC
CTGGAACTCATGCAGTACCTTCCTCAGCACACAATTGAAACGTACAGGCAACAGCAACAGCAGCAGCACC
AGCACTTACTTCAGAAACAGACCTCAATACAGTCTCCATCTTCATATGGTAACAGCTCCCCACCTCTGAA
CAAAATGAACAGCATGAACAAGCTGCCTTCTGTGAGCCAGCTTATCAACCCTCAGCAGCGCAACGCCCTC
ACTCCTACAACCATTCCTGATGGCATGGGAGCCAACATTCCCATGATGGGCACCCACATGCCAATGGCTG
GAGACATGAATGGACTCAGCCCCACCCAGGCACTCCCTCCCCCACTCTCCATGCCATCCACCTCCCACTG
CACACCCCCACCTCCGTATCCCACAGATTGCAGCATTGTCAGTTTCTTAGCGAGGTTGGGCTGTTCATCA
TGTCTGGACTATTTCACGACCCAGGGGCTGACCACCATCTATCAGATTGAGCATTACTCCATGGATGATC
TGGCAAGTCTGAAAATCCCTGAGCAATTTCGACATGCGATCTGGAAGGGCATCCTGGACCACCGGCAGCT
CCACGAATTCTCCTCCCCTTCTCATCTCCTGCGGACCCCAAGCAGTGCCTCTACAGTCAGTGTGGGCTCC
AGTGAGACCCGGGGTGAGCGTGTTATTGATGCTGTGCGATTCACCCTCCGCCAGACCATCTCTTTCCCAC
CCCGAGATGAGTGGAATGACTTCAACTTTGACATGGATGCTCGCCGCAATAAGCAACAGCGCATCAAAGA
GGAGGGGGAGTGA'''.replace('\n','')

In [28]:
TP63_flex=flex_padlock(TP63_CDS,'c.1090G>A')
TP63_flex

{'LHS_mut': 'GCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCACACCGTTCTTTGTACTGTCCGAAAT',
 'LHS_ref': 'GCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCACACCGTTCTTTGTACTGTCCGAAAC',
 'RHS': '/5Phos/TTGCTGCTTTCTGATGCTATCTTCAACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAA',
 'padlock_alt': '/5Phos/TTGCTGCTTTCTGATGCTATCTTCAACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCACACCGTTCTTTGTACTGTCCGAAAT',
 'padlock_ref': '/5Phos/TTGCTGCTTTCTGATGCTATCTTCAACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCACACCGTTCTTTGTACTGTCCGAAAC',
 'target_LHS_mut': 'CACCGTTCTTTGTACTGTCCGAAAT',
 'target_LHS_ref': 'CACCGTTCTTTGTACTGTCCGAAAC',
 'target_RHS': 'TTGCTGCTTTCTGATGCTATCTTCA'}

In [27]:
FGFR1_CDS='''ATGGAGGCAAGGGTCAGTTTGAAAAGGAGGATCGAGCTCACTGTGGAGTATCCATGGAGATGTGGAGCCT
TGTCACCAACCTCTAACTGCAGAACTGGGATGTGGAGCTGGAAGTGCCTCCTCTTCTGGGCTGTGCTGGT
CACAGCCACACTCTGCACCGCTAGGCCGTCCCCGACCTTGCCTGAACAAGCCCAGCCCTGGGGAGCCCCT
GTGGAAGTGGAGTCCTTCCTGGTCCACCCCGGTGACCTGCTGCAGCTTCGCTGTCGGCTGCGGGACGATG
TGCAGAGCATCAACTGGCTGCGGGACGGGGTGCAGCTGGCGGAAAGCAACCGCACCCGCATCACAGGGGA
GGAGGTGGAGGTGCAGGACTCCGTGCCCGCAGACTCCGGCCTCTATGCTTGCGTAACCAGCAGCCCCTCG
GGCAGTGACACCACCTACTTCTCCGTCAATGTTTCAGATGCTCTCCCCTCCTCGGAGGATGATGATGATG
ATGATGACTCCTCTTCAGAGGAGAAAGAAACAGATAACACCAAACCAAACCCCGTAGCTCCATATTGGAC
ATCCCCAGAAAAGATGGAAAAGAAATTGCATGCAGTGCCGGCTGCCAAGACAGTGAAGTTCAAATGCCCT
TCCAGTGGGACCCCAAACCCCACACTGCGCTGGTTGAAAAATGGCAAAGAATTCAAACCTGACCACAGAA
TTGGAGGCTACAAGGTCCGTTATGCCACCTGGAGCATCATAATGGACTCTGTGGTGCCCTCTGACAAGGG
CAACTACACCTGCATTGTGGAGAATGAGTACGGCAGCATCAACCACACATACCAGCTGGATGTCGTGGAG
CGGTCCCCTCACCGGCCCATCCTGCAAGCAGGGTTGCCCGCCAACAAAACAGTGGCCCTGGGTAGCAACG
TGGAGTTCATGTGTAAGGTGTACAGTGACCCGCAGCCGCACATCCAGTGGCTAAAGCACATCGAGGTGAA
TGGGAGCAAGATTGGCCCAGACAACCTGCCTTATGTCCAGATCTTGAAGACTGCTGGAGTTAATACCACC
GACAAAGAGATGGAGGTGCTTCACTTAAGAAATGTCTCCTTTGAGGACGCAGGGGAGTATACGTGCTTGG
CGGGTAACTCTATCGGACTCTCCCATCACTCTGCATGGTTGACCGTTCTGGAAGCCCTGGAAGAGAGGCC
GGCAGTGATGACCTCGCCCCTGTACCTGGAGATCATCATCTATTGCACAGGGGCCTTCCTCATCTCCTGC
ATGGTGGGGTCGGTCATCGTCTACAAGATGAAGAGTGGTACCAAGAAGAGTGACTTCCACAGCCAGATGG
CTGTGCACAAGCTGGCCAAGAGCATCCCTCTGCGCAGACAGGTAACAGTGTCTGCTGACTCCAGTGCATC
CATGAACTCTGGGGTTCTTCTGGTTCGGCCATCACGGCTCTCCTCCAGTGGGACTCCCATGCTAGCAGGG
GTCTCTGAGTATGAGCTTCCCGAAGACCCTCGCTGGGAGCTGCCTCGGGACAGACTGGTCTTAGGCAAAC
CCCTGGGAGAGGGCTGCTTTGGGCAGGTGGTGTTGGCAGAGGCTATCGGGCTGGACAAGGACAAACCCAA
CCGTGTGACCAAAGTGGCTGTGAAGATGTTGAAGTCGGACGCAACAGAGAAAGACTTGTCAGACCTGATC
TCAGAAATGGAGATGATGAAGATGATCGGGAAGCATAAGAATATCATCAACCTGCTGGGGGCCTGCACGC
AGGATGGTCCCTTGTATGTCATCGTGGAGTATGCCTCCAAGGGCAACCTGCGGGAGTACCTGCAGGCCCG
GAGGCCCCCAGGGCTGGAATACTGCTACAACCCCAGCCACAACCCAGAGGAGCAGCTCTCCTCCAAGGAC
CTGGTGTCCTGCGCCTACCAGGTGGCCCGAGGCATGGAGTATCTGGCCTCCAAGAAGTGCATACACCGAG
ACCTGGCAGCCAGGAATGTCCTGGTGACAGAGGACAATGTGATGAAGATAGCAGACTTTGGCCTCGCACG
GGACATTCACCACATCGACTACTATAAAAAGACAACCAACGGCCGACTGCCTGTGAAGTGGATGGCACCC
GAGGCATTATTTGACCGGATCTACACCCACCAGAGTGATGTGTGGTCTTTCGGGGTGCTCCTGTGGGAGA
TCTTCACTCTGGGCGGCTCCCCATACCCCGGTGTGCCTGTGGAGGAACTTTTCAAGCTGCTGAAGGAGGG
TCACCGCATGGACAAGCCCAGTAACTGCACCAACGAGCTGTACATGATGATGCGGGACTGCTGGCATGCA
GTGCCCTCACAGAGACCCACCTTCAAGCAGCTGGTGGAAGACCTGGACCGCATCGTGGCCTTGACCTCCA
ACCAGGAGTACCTGGACCTGTCCATGCCCCTGGACCAGTACTCCCCCAGCTTTCCCGACACCCGGAGCTC
TACGTGCTCCTCAGGGGAGGATTCCGTCTTCTCTCATGAGCCGCTGCCCGAGGAGCCCTGCCTGCCCCGA
CACCCAGCCCAGCTTGCCAATGGCGGACTCAAACGCCGCTGA'''.replace('\n','')

In [30]:
FGFR1_flex=flex_padlock(FGFR1_CDS, "c.719G>A")
FGFR1_flex

{'LHS_mut': 'GCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCAATTATGATGCTCCAGGTGGCATAAT',
 'LHS_ref': 'GCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCAATTATGATGCTCCAGGTGGCATAAC',
 'RHS': '/5Phos/GGACCTTGTAGCCTCCAATTCTGTGACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAA',
 'padlock_alt': '/5Phos/GGACCTTGTAGCCTCCAATTCTGTGACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCAATTATGATGCTCCAGGTGGCATAAT',
 'padlock_ref': '/5Phos/GGACCTTGTAGCCTCCAATTCTGTGACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCAATTATGATGCTCCAGGTGGCATAAC',
 'target_LHS_mut': 'ATTATGATGCTCCAGGTGGCATAAT',
 'target_LHS_ref': 'ATTATGATGCTCCAGGTGGCATAAC',
 'target_RHS': 'GGACCTTGTAGCCTCCAATTCTGTG'}

In [31]:
### https://cancer.sanger.ac.uk/cosmic/mutation/overview?id=130818805

In [28]:
PTPRC='''ATGACCATGTATTTGTGGCTTAAACTCTTGGCATTTGGCTTTGCCTTTCTGGACACAGAAGTATTTGTGA
CAGGGCAAAGCCCAACACCTTCCCCCACTGGATTGACTACAGCAAAGATGCCCAGTGTTCCACTTTCAAG
TGACCCCTTACCTACTCACACCACTGCATTCTCACCCGCAAGCACCTTTGAAAGAGAAAATGACTTCTCA
GAGACCACAACTTCTCTTAGTCCAGACAATACTTCCACCCAAGTATCCCCGGACTCTTTGGATAATGCTA
GTGCTTTTAATACCACAGGTGTTTCATCAGTACAGACGCCTCACCTTCCCACGCACGCAGACTCGCAGAC
GCCCTCTGCTGGAACTGACACGCAGACATTCAGCGGCTCCGCCGCCAATGCAAAACTCAACCCTACCCCA
GGCAGCAATGCTATCTCAGATGTCCCAGGAGAGAGGAGTACAGCCAGCACCTTTCCTACAGACCCAGTTT
CCCCATTGACAACCACCCTCAGCCTTGCACACCACAGCTCTGCTGCCTTACCTGCACGCACCTCCAACAC
CACCATCACAGCGAACACCTCAGATGCCTACCTTAATGCCTCTGAAACAACCACTCTGAGCCCTTCTGGA
AGCGCTGTCATTTCAACCACAACAATAGCTACTACTCCATCTAAGCCAACATGTGATGAAAAATATGCAA
ACATCACTGTGGATTACTTATATAACAAGGAAACTAAATTATTTACAGCAAAGCTAAATGTTAATGAGAA
TGTGGAATGTGGAAACAATACTTGCACAAACAATGAGGTGCATAACCTTACAGAATGTAAAAATGCGTCT
GTTTCCATATCTCATAATTCATGTACTGCTCCTGATAAGACATTAATATTAGATGTGCCACCAGGGGTTG
AAAAGTTTCAGTTACATGATTGTACACAAGTTGAAAAAGCAGATACTACTATTTGTTTAAAATGGAAAAA
TATTGAAACCTTTACTTGTGATACACAGAATATTACCTACAGATTTCAGTGTGGTAATATGATATTTGAT
AATAAAGAAATTAAATTAGAAAACCTTGAACCCGAACATGAGTATAAGTGTGACTCAGAAATACTCTATA
ATAACCACAAGTTTACTAACGCAAGTAAAATTATTAAAACAGATTTTGGGAGTCCAGGAGAGCCTCAGAT
TATTTTTTGTAGAAGTGAAGCTGCACATCAAGGAGTAATTACCTGGAATCCCCCTCAAAGATCATTTCAT
AATTTTACCCTCTGTTATATAAAAGAGACAGAAAAAGATTGCCTCAATCTGGATAAAAACCTGATCAAAT
ATGATTTGCAAAATTTAAAACCTTATACGAAATATGTTTTATCATTACATGCCTACATCATTGCAAAAGT
GCAACGTAATGGAAGTGCTGCAATGTGTCATTTCACAACTAAAAGTGCTCCTCCAAGCCAGGTCTGGAAC
ATGACTGTCTCCATGACATCAGATAATAGTATGCATGTCAAGTGTAGGCCTCCCAGGGACCGTAATGGCC
CCCATGAACGTTACCATTTGGAAGTTGAAGCTGGAAATACTCTGGTTAGAAATGAGTCGCATAAGAATTG
CGATTTCCGTGTAAAAGATCTTCAATATTCAACAGACTACACTTTTAAGGCCTATTTTCACAATGGAGAC
TATCCTGGAGAACCCTTTATTTTACATCATTCAACATCTTATAATTCTAAGGCACTGATAGCATTTCTGG
CATTTCTGATTATTGTGACATCAATAGCCCTGCTTGTTGTTCTCTACAAAATCTATGATCTACATAAGAA
AAGATCCTGCAATTTAGATGAACAGCAGGAGCTTGTTGAAAGGGATGATGAAAAACAACTGATGAATGTG
GAGCCAATCCATGCAGATATTTTGTTGGAAACTTATAAGAGGAAGATTGCTGATGAAGGAAGACTTTTTC
TGGCTGAATTTCAGAGCATCCCGCGGGTGTTCAGCAAGTTTCCTATAAAGGAAGCTCGAAAGCCCTTTAA
CCAGAATAAAAACCGTTATGTTGACATTCTTCCTTATGATTATAACCGTGTTGAACTCTCTGAGATAAAC
GGAGATGCAGGGTCAAACTACATAAATGCCAGCTATATTGATGGTTTCAAAGAACCCAGGAAATACATTG
CTGCACAAGGTCCCAGGGATGAAACTGTTGATGATTTCTGGAGGATGATTTGGGAACAGAAAGCCACAGT
TATTGTCATGGTCACTCGATGTGAAGAAGGAAACAGGAACAAGTGTGCAGAATACTGGCCGTCAATGGAA
GAGGGCACTCGGGCTTTTGGAGATGTTGTTGTAAAGATCAACCAGCACAAAAGATGTCCAGATTACATCA
TTCAGAAATTGAACATTGTAAATAAAAAAGAAAAAGCAACTGGAAGAGAGGTGACTCACATTCAGTTCAC
CAGCTGGCCAGACCACGGGGTGCCTGAGGATCCTCACTTGCTCCTCAAACTGAGAAGGAGAGTGAATGCC
TTCAGCAATTTCTTCAGTGGTCCCATTGTGGTGCACTGCAGTGCTGGTGTTGGGCGCACAGGAACCTATA
TCGGAATTGATGCCATGCTAGAAGGCCTGGAAGCCGAGAACAAAGTGGATGTTTATGGTTATGTTGTCAA
GCTAAGGCGACAGAGATGCCTGATGGTTCAAGTAGAGGCCCAGTACATCTTGATCCATCAGGCTTTGGTG
GAATACAATCAGTTTGGAGAAACAGAAGTGAATTTGTCTGAATTACATCCATATCTACATAACATGAAGA
AAAGGGATCCACCCAGTGAGCCGTCTCCACTAGAGGCTGAATTCCAGAGACTTCCTTCATATAGGAGCTG
GAGGACACAGCACATTGGAAATCAAGAAGAAAATAAAAGTAAAAACAGGAATTCTAATGTCATCCCATAT
GACTATAACAGAGTGCCACTTAAACATGAGCTGGAAATGAGTAAAGAGAGTGAGCATGATTCAGATGAAT
CCTCTGATGATGACAGTGATTCAGAGGAACCAAGCAAATACATCAATGCATCTTTTATAATGAGCTACTG
GAAACCTGAAGTGATGATTGCTGCTCAGGGACCACTGAAGGAGACCATTGGTGACTTTTGGCAGATGATC
TTCCAAAGAAAAGTCAAAGTTATTGTTATGCTGACAGAACTGAAACATGGAGACCAGGAAATCTGTGCTC
AGTACTGGGGAGAAGGAAAGCAAACATATGGAGATATTGAAGTTGACCTGAAAGACACAGACAAATCTTC
AACTTATACCCTTCGTGTCTTTGAACTGAGACATTCCAAGAGGAAAGACTCTCGAACTGTGTACCAGTAC
CAATATACAAACTGGAGTGTGGAGCAGCTTCCTGCAGAACCCAAGGAATTAATCTCTATGATTCAGGTCG
TCAAACAAAAACTTCCCCAGAAGAATTCCTCTGAAGGGAACAAGCATCACAAGAGTACACCTCTACTCAT
TCACTGCAGGGATGGATCTCAGCAAACGGGAATATTTTGTGCTTTGTTAAATCTCTTAGAAAGTGCGGAA
ACAGAAGAGGTAGTGGATATTTTTCAAGTGGTAAAAGCTCTACGCAAAGCTAGGCCAGGCATGGTTTCCA
CATTCGAGCAATATCAATTCCTATATGACGTCATTGCCAGCACCTACCCTGCTCAGAATGGACAAGTAAA
GAAAAACAACCATCAAGAAGATAAAATTGAATTTGATAATGAAGTGGACAAAGTAAAGCAGGATGCTAAT
TGTGTTAATCCACTTGGTGCCCCAGAAAAGCTCCCTGAAGCAAAGGAACAGGCTGAAGGTTCTGAACCCA
CGAGTGGCACTGAGGGGCCAGAACATTCTGTCAATGGTCCTGCAAGTCCAGCTTTAAATCAAGGTTCATA
G'''.replace('\n','')

COSMIC: p.M769I; "c.2307G>A"
Bing: M769K, T>A, "c.2306T>A"

In [33]:
PTPRC_flex=flex_padlock(PTPRC, "c.2306T>A")
PTPRC_flex

{'LHS_mut': 'GCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCACCAAAAGCCCGAGTGCCCTCTTCCT',
 'LHS_ref': 'GCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCACCAAAAGCCCGAGTGCCCTCTTCCA',
 'RHS': '/5Phos/TTGACGGCCAGTATTCTGCACACTTACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAA',
 'padlock_alt': '/5Phos/TTGACGGCCAGTATTCTGCACACTTACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCACCAAAAGCCCGAGTGCCCTCTTCCT',
 'padlock_ref': '/5Phos/TTGACGGCCAGTATTCTGCACACTTACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCACCAAAAGCCCGAGTGCCCTCTTCCA',
 'target_LHS_mut': 'CCAAAAGCCCGAGTGCCCTCTTCCT',
 'target_LHS_ref': 'CCAAAAGCCCGAGTGCCCTCTTCCA',
 'target_RHS': 'TTGACGGCCAGTATTCTGCACACTT'}

In [35]:
PTPRC_flex=flex_padlock(PTPRC, "c.2306T>A")
PTPRC_flex

{'LHS_mut': 'GCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCACCAAAAGCCCGAGTGCCCTCTTCCT',
 'LHS_ref': 'GCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCACCAAAAGCCCGAGTGCCCTCTTCCA',
 'RHS': '/5Phos/TTGACGGCCAGTATTCTGCACACTTACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAA',
 'padlock_alt': '/5Phos/TTGACGGCCAGTATTCTGCACACTTACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCACCAAAAGCCCGAGTGCCCTCTTCCT',
 'padlock_ref': '/5Phos/TTGACGGCCAGTATTCTGCACACTTACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCACCAAAAGCCCGAGTGCCCTCTTCCA',
 'target_LHS_mut': 'CCAAAAGCCCGAGTGCCCTCTTCCT',
 'target_LHS_ref': 'CCAAAAGCCCGAGTGCCCTCTTCCA',
 'target_RHS': 'TTGACGGCCAGTATTCTGCACACTT'}

# order probes in 200uM DDW, hplc if possible

# entrez

https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.EFetch

# get human cds

In [50]:
def get_cds(gene,organism='human'):
    records=[]
    #query ids
    handle = Entrez.esearch(db="nuccore", term="{}[Orgn] AND {}[Gene]".format(organism,gene))
    record = Entrez.read(handle)
    for idx in record["IdList"]:
        try:
            Entrez.email = "A.N.Other@example.com"     # Always tell NCBI who you are
            handle = Entrez.efetch(db="nuccore", id=idx, rettype="fasta_cds_na", retmode="text")
            filename = gene+str(idx)+".fasta"
            if not os.path.isfile(filename):
                with open(filename, "w") as out_handle:
                    out_handle.write(handle.read())
                print(gene,idx," Saved")

                print("Parsing...")
                record = SeqIO.read(filename, "fasta")
                records.append(record)
        except:
            continue
    return records

In [51]:
DNAJB1=get_cds("DNAJB1")

In [29]:
DNAJB1_CDS='''
ATGGGTAAAGACTACTACCAGACGTTGGGCCTGGCCCGCGGCGCGTCGGACGAGGAGATCAAGCGGGCCT
ACCGCCGCCAGGCGCTGCGCTACCACCCGGACAAGAACAAGGAGCCCGGCGCCGAGGAGAAGTTCAAGGA
GATCGCTGAGGCCTACGACGTGCTCAGCGACCCGCGCAAGCGCGAGATCTTCGACCGCTACGGGGAGGAA
GGCCTAAAGGGGAGTGGCCCCAGTGGCGGTAGCGGCGGTGGTGCCAATGGTACCTCTTTCAGCTACACAT
TCCATGGAGACCCTCATGCCATGTTTGCTGAGTTCTTCGGTGGCAGAAATCCCTTTGACACCTTTTTTGG
GCAGCGGAACGGGGAGGAAGGCATGGACATTGATGACCCATTCTCTGGCTTCCCTATGGGCATGGGTGGC
TTCACCAACGTGAACTTTGGCCGCTCCCGCTCTGCCCAAGAGCCCGCCCGAAAGAAGCAAGATCCCCCAG
TCACCCACGACCTTCGAGTCTCCCTTGAAGAGATCTACAGCGGCTGTACCAAGAAGATGAAAATCTCCCA
CAAGCGGCTAAACCCCGACGGAAAGAGCATTCGAAACGAAGACAAAATATTGACCATCGAAGTGAAGAAG
GGGTGGAAAGAAGGAACCAAAATCACTTTCCCCAAGGAAGGAGACCAGACCTCCAACAACATTCCAGCTG
ATATCGTCTTTGTTTTAAAGGACAAGCCCCACAATATCTTTAAGAGAGATGGCTCTGATGTCATTTATCC
TGCCAGGATCAGCCTCCGGGAGGCTCTGTGTGGCTGCACAGTGAACGTCCCCACTCTGGACGGCAGGACG
ATACCCGTCGTATTCAAAGATGTTATCAGGCCTGGCATGCGGCGAAAAGTTCCTGGAGAAGGCCTCCCCC
TCCCCAAAACACCCGAGAAACGTGGGGACCTCATTATTGAGTTTGAAGTGATCTTCCCCGAAAGGATTCC
CCAGACATCAAGAACCGTACTTGAGCAGGTTCTTCCAATATAG'''.replace('\n','')

In [47]:
# for r in DNAJB1:
#     if 'CCDS12312.1' in r.description:
#         DNAJB1_CDS=r.seq

In [62]:
DNAJB1_CDS

'ATGGGTAAAGACTACTACCAGACGTTGGGCCTGGCCCGCGGCGCGTCGGACGAGGAGATCAAGCGGGCCTACCGCCGCCAGGCGCTGCGCTACCACCCGGACAAGAACAAGGAGCCCGGCGCCGAGGAGAAGTTCAAGGAGATCGCTGAGGCCTACGACGTGCTCAGCGACCCGCGCAAGCGCGAGATCTTCGACCGCTACGGGGAGGAAGGCCTAAAGGGGAGTGGCCCCAGTGGCGGTAGCGGCGGTGGTGCCAATGGTACCTCTTTCAGCTACACATTCCATGGAGACCCTCATGCCATGTTTGCTGAGTTCTTCGGTGGCAGAAATCCCTTTGACACCTTTTTTGGGCAGCGGAACGGGGAGGAAGGCATGGACATTGATGACCCATTCTCTGGCTTCCCTATGGGCATGGGTGGCTTCACCAACGTGAACTTTGGCCGCTCCCGCTCTGCCCAAGAGCCCGCCCGAAAGAAGCAAGATCCCCCAGTCACCCACGACCTTCGAGTCTCCCTTGAAGAGATCTACAGCGGCTGTACCAAGAAGATGAAAATCTCCCACAAGCGGCTAAACCCCGACGGAAAGAGCATTCGAAACGAAGACAAAATATTGACCATCGAAGTGAAGAAGGGGTGGAAAGAAGGAACCAAAATCACTTTCCCCAAGGAAGGAGACCAGACCTCCAACAACATTCCAGCTGATATCGTCTTTGTTTTAAAGGACAAGCCCCACAATATCTTTAAGAGAGATGGCTCTGATGTCATTTATCCTGCCAGGATCAGCCTCCGGGAGGCTCTGTGTGGCTGCACAGTGAACGTCCCCACTCTGGACGGCAGGACGATACCCGTCGTATTCAAAGATGTTATCAGGCCTGGCATGCGGCGAAAAGTTCCTGGAGAAGGCCTCCCCCTCCCCAAAACACCCGAGAAACGTGGGGACCTCATTATTGAGTTTGAAGTGATCTTCCCCGAAAGGATTCCCCAGACATCAAGAACCGTA

In [63]:
DNAJB1_CDS[555:186*3]


'TCC'

In [64]:
# control probes
# DNAJB1, S186S, 
DNAJB1_flex=flex_padlock(DNAJB1_CDS, "c.558C>A")

In [65]:
DNAJB1_flex

{'LHS_mut': 'GCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCATCCGTCGGGGTTTAGCCGCTTGTGT',
 'LHS_ref': 'GCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCATCCGTCGGGGTTTAGCCGCTTGTGG',
 'RHS': '/5Phos/GAGATTTTCATCTTCTTGGTACAGCACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAA',
 'padlock_alt': '/5Phos/GAGATTTTCATCTTCTTGGTACAGCACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCATCCGTCGGGGTTTAGCCGCTTGTGT',
 'padlock_ref': '/5Phos/GAGATTTTCATCTTCTTGGTACAGCACGCGGTTAGCACGTANNATACGTCACGGTCCTAGCAAGCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCATCCGTCGGGGTTTAGCCGCTTGTGG',
 'target_LHS_mut': 'TCCGTCGGGGTTTAGCCGCTTGTGT',
 'target_LHS_ref': 'TCCGTCGGGGTTTAGCCGCTTGTGG',
 'target_RHS': 'GAGATTTTCATCTTCTTGGTACAGC'}

In [ ]:
# write to file

In [66]:
KRAS_flex=flex_padlock(KRAS_CDS,'c.34G>T')
FGFR1_flex=flex_padlock(FGFR1_CDS, "c.719G>A")
TP63_flex=flex_padlock(TP63_CDS,'c.1090G>A')
PTPRC_flex=flex_padlock(PTPRC, "c.2306T>A")
DNAJB1_flex=flex_padlock(DNAJB1_CDS, "c.558C>A")

In [30]:
targets={'KRAS':{'CDS':KRAS_CDS,'mutation_list':['c.34G>T',]},
         'FGFR1':{'CDS':FGFR1_CDS, 'mutation_list':["c.719G>A",]},
         'TP63':{'CDS':TP63_CDS,'mutation_list':['c.1090G>A',]},
         "PTPRC":{'CDS':PTPRC, 'mutation_list':["c.2306T>A",]},
         "DNAJB1":{'CDS':DNAJB1_CDS, 'mutation_list':["c.558C>A",]},
          }

In [31]:
output=dict()
for gene, info in targets.items():
    CDS=info['CDS']
    mutation_list=info['mutation_list']
    for mut in mutation_list:
        output[gene+"_"+mut]=flex_padlock_gap(CDS, mut)
df=pd.DataFrame.from_dict(output,orient='index')


In [81]:
# dd_probe=dict()
# for k, i in output.items():
#     if i['LHS_mut'][:-1]==i['LHS_ref'][:-1]:
#         i['ddLHS']=i['LHS_mut'][:-1]
#     if i['padlock_alt'][:-1]==i['padlock_ref'][:-1]:
#         i['ddpadlock']=i['padlock_alt'][:-1] 
#     i['ddRHS']=i['RHS']
#     dd_probe[k]=i
    


In [32]:
df

,LHS,target_LHS_ref,target_LHS_mut,target_RHS,RHS,padlock
KRAS_c.34G>T,GCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCAGTCAA...,GTCAAGGCACTCTTGCCTACGCCACC,GTCAAGGCACTCTTGCCTACGCCACA,AGCTCCAACTACCACAAGTTTATATTC,/5Phos/CTCCAACTACCACAAGTTTATATTCACGCGGTTAGCACG...,/5Phos/CTCCAACTACCACAAGTTTATATTCACGCGGTTAGCACG...
FGFR1_c.719G>A,GCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCACATTA...,CATTATGATGCTCCAGGTGGCATAAC,CATTATGATGCTCCAGGTGGCATAAT,GGACCTTGTAGCCTCCAATTCTGTGGT,/5Phos/ACCTTGTAGCCTCCAATTCTGTGGTACGCGGTTAGCACG...,/5Phos/ACCTTGTAGCCTCCAATTCTGTGGTACGCGGTTAGCACG...
TP63_c.1090G>A,GCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCATCACC...,TCACCGTTCTTTGTACTGTCCGAAAC,TCACCGTTCTTTGTACTGTCCGAAAT,TTGCTGCTTTCTGATGCTATCTTCATC,/5Phos/GCTGCTTTCTGATGCTATCTTCATCACGCGGTTAGCACG...,/5Phos/GCTGCTTTCTGATGCTATCTTCATCACGCGGTTAGCACG...
PTPRC_c.2306T>A,GCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCATCCAA...,TCCAAAAGCCCGAGTGCCCTCTTCCA,TCCAAAAGCCCGAGTGCCCTCTTCCT,TTGACGGCCAGTATTCTGCACACTTGT,/5Phos/GACGGCCAGTATTCTGCACACTTGTACGCGGTTAGCACG...,/5Phos/GACGGCCAGTATTCTGCACACTTGTACGCGGTTAGCACG...
DNAJB1_c.558C>A,GCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCATTCCG...,TTCCGTCGGGGTTTAGCCGCTTGTGG,TTCCGTCGGGGTTTAGCCGCTTGTGT,GAGATTTTCATCTTCTTGGTACAGCCG,/5Phos/GATTTTCATCTTCTTGGTACAGCCGACGCGGTTAGCACG...,/5Phos/GATTTTCATCTTCTTGGTACAGCCGACGCGGTTAGCACG...


In [34]:
# df=pd.DataFrame.from_dict(dd_probe,orient='index')
df.to_csv('flex_dd_padlock_3gap_231117.csv')

In [35]:
to_order={}
for k, i in df.items():
    for kk,ii in i.items():
        to_order[kk+'_'+k]=ii
ddf=pd.DataFrame.from_dict(to_order,orient="index")
ddf['len']=ddf[0].apply(len)
ddf.sort_index(inplace=True)
ddf.to_csv('flex_dd_padlock_3gap_231117_toorder.csv')

In [36]:
ddf

,0,len
DNAJB1_c.558C>A_LHS,GCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCATTCCG...,66
DNAJB1_c.558C>A_RHS,/5Phos/GATTTTCATCTTCTTGGTACAGCCGACGCGGTTAGCACG...,70
DNAJB1_c.558C>A_padlock,/5Phos/GATTTTCATCTTCTTGGTACAGCCGACGCGGTTAGCACG...,136
DNAJB1_c.558C>A_target_LHS_mut,TTCCGTCGGGGTTTAGCCGCTTGTGT,26
DNAJB1_c.558C>A_target_LHS_ref,TTCCGTCGGGGTTTAGCCGCTTGTGG,26
DNAJB1_c.558C>A_target_RHS,GAGATTTTCATCTTCTTGGTACAGCCG,27
FGFR1_c.719G>A_LHS,GCAACGACTCCACACTCATACCTTGGCACCCGAGAATTCCACATTA...,66
FGFR1_c.719G>A_RHS,/5Phos/ACCTTGTAGCCTCCAATTCTGTGGTACGCGGTTAGCACG...,70
FGFR1_c.719G>A_padlock,/5Phos/ACCTTGTAGCCTCCAATTCTGTGGTACGCGGTTAGCACG...,136
FGFR1_c.719G>A_target_LHS_mut,CATTATGATGCTCCAGGTGGCATAAT,26


# 3 quick data analysis

In [4]:
ref=pd.read_csv('flex_dna_probes_230300.csv')

In [5]:
# ref.to_csv('flex_dna_230310_ref.csv')

In [6]:
ref['target_LHS_ref'][2]

'TCAAGGCACTCTTGCCTACGCCACC'

In [7]:
ref['amplicon_ref']=ref['target_LHS_ref']+ref['target_RHS']
ref['amplicon_mut']=ref['target_LHS_mut']+ref['target_RHS']
# ref['cds_ref']=ref['target_LHS_mut']+ref['target_RHS']

In [8]:
Seq("TCAAGGCACTCTTGCCTACGCCACC").reverse_complement()

Seq('GGTGGCGTAGGCAAGAGTGCCTTGA')

In [10]:
target_dict={}
for i,r in ref.iterrows():
    
    gene=r['Unnamed: 0']
    refs=r['amplicon_ref']
    muts=r['amplicon_mut']
    target_dict[gene+'_wt']=refs
    target_dict[gene+'_mt']=muts
    


In [10]:
# fastq= '/gstore/data/dnaseq/processed_runs/R9173/results/LIB5468564_HITS5485163_S4_L001_R2_001.fastq.gz'

In [11]:
# fastq= '/gstore/home/taol9/repeats_ultima/LIB5468564_HITS5485163_S4_L001_R2_001.fastq'

In [12]:
# fastq=  "/gstore/data/ctgbioinfo/taol9/flex/25/LIB5469834_SAM24431961_S5_L002_R2_001.fastq"

In [17]:
# fastq=  "/gstore/data/ctgbioinfo/taol9/flex/40/LIB5472587_SAM24434098_S1_L001_R2_001.fastq"  # linear probe


In [18]:
from Bio import SeqIO
record_dict = SeqIO.index(fastq,'fastq')
len(record_dict)

42081106

In [14]:
len(record_dict)

15507195

In [12]:
def mapper(k):
    ontarget=0
    parsedreads=0
    aligns=list()
    for i in list(record_dict.keys())[:1000000]:
        record=record_dict[i]
        seq=record.seq
        tar=target_dict[k]
        if ontarget < 100000:
            parsedreads+=1
            if str(tar) in str(seq):
                aligns.append(record.id)
                ontarget+=1   
            else:
                continue
    print (k, ontarget, parsedreads)
    return {k:aligns}
    

In [18]:
keys=list(target_dict.keys())
output1={}
for k in keys:
    x=mapper(k)
    output1.update(x)
#     ontarget=0
#     parsedreads=0
#     for i in list(record_dict.keys())[:100000]:
#         record=record_dict[i]
#         seq=record.seq
#         tar=target_dict[k]
#         if ontarget < 3 or parsedreads < 100000:
#             parsedreads+=1
#             if str(tar) in str(seq):
#                 aligns[k].append(record.id)
#                 ontarget+=1   
#             else:
#                 continue
#     print (k, ontarget,parsedreads)
    

DNAJB1_c.558C>A_wt 53 1500000
DNAJB1_c.558C>A_mt 50 1500000
FGFR1_c.719G>A_wt 0 1500000
FGFR1_c.719G>A_mt 2 1500000
KRAS_c.34G>T_wt 32 1500000
KRAS_c.34G>T_mt 10 1500000
PTPRC_c.2306T>A_wt 2 1500000
PTPRC_c.2306T>A_mt 0 1500000
TP63_c.1090G>A_wt 36 1500000
TP63_c.1090G>A_mt 1 1500000


In [ ]:
#25 cyc
DNAJB1_c.558C>A_wt 53 1500000
DNAJB1_c.558C>A_mt 50 1500000
FGFR1_c.719G>A_wt 0 1500000
FGFR1_c.719G>A_mt 2 1500000
KRAS_c.34G>T_wt 32 1500000
KRAS_c.34G>T_mt 10 1500000
PTPRC_c.2306T>A_wt 2 1500000
PTPRC_c.2306T>A_mt 0 1500000
TP63_c.1090G>A_wt 36 1500000
TP63_c.1090G>A_mt 1 1500000

In [22]:
#40C linear
keys=list(target_dict.keys())
output1={}
for k in keys:
    x=mapper(k)
    output1.update(x)

DNAJB1_c.558C>A_wt 100 2046
DNAJB1_c.558C>A_mt 0 1500000
FGFR1_c.719G>A_wt 34 1500000
FGFR1_c.719G>A_mt 0 1500000
KRAS_c.34G>T_wt 100 786
KRAS_c.34G>T_mt 100 244874
PTPRC_c.2306T>A_wt 100 1040910
PTPRC_c.2306T>A_mt 0 1500000
TP63_c.1090G>A_wt 100 1611
TP63_c.1090G>A_mt 100 401705


In [13]:
# 40 padlock
fastq=  "/gstore/data/ctgbioinfo/taol9/flex/40/LIB5472588_SAM24434099_S2_L001_R2_001.fastq"  # padloc probe
record_dict = SeqIO.index(fastq,'fastq')
print (len(record_dict))
keys=list(target_dict.keys())
output1={}
for k in keys:
    x=mapper(k)
    output1.update(x)
with open('output1_40_padlock.pkl', 'wb') as file:
    pickle.dump(output1, file)

41056602
DNAJB1_c.558C>A_wt 59652 1000000
DNAJB1_c.558C>A_mt 13479 1000000
FGFR1_c.719G>A_wt 30 1000000
FGFR1_c.719G>A_mt 20780 1000000
KRAS_c.34G>T_wt 70694 1000000
KRAS_c.34G>T_mt 2706 1000000
PTPRC_c.2306T>A_wt 6035 1000000
PTPRC_c.2306T>A_mt 683 1000000
TP63_c.1090G>A_wt 24380 1000000
TP63_c.1090G>A_mt 6410 1000000


In [ ]:
import pickle
with open('output1_25.pkl', 'wb') as file:
    pickle.dump(output1, file)

In [27]:
keys=list(target_dict.keys())
output1={}
for k in keys:
    x=mapper(k)
    output1.update(x)
import pickle
with open('output1_40.pkl', 'wb') as file:
    pickle.dump(output1, file)

DNAJB1_c.558C>A_wt 37722 1000000
DNAJB1_c.558C>A_mt 0 1000000
FGFR1_c.719G>A_wt 25 1000000
FGFR1_c.719G>A_mt 0 1000000
KRAS_c.34G>T_wt 100000 803446
KRAS_c.34G>T_mt 436 1000000
PTPRC_c.2306T>A_wt 98 1000000
PTPRC_c.2306T>A_mt 0 1000000
TP63_c.1090G>A_wt 55697 1000000
TP63_c.1090G>A_mt 237 1000000


In [ ]:
DNAJB1_c.558C>A_wt 37722 1000000
DNAJB1_c.558C>A_mt 0 1000000
KRAS_c.34G>T_wt 100000 803446
KRAS_c.34G>T_mt 436 1000000
PTPRC_c.2306T>A_wt 98 1000000
PTPRC_c.2306T>A_mt 0 1000000
TP63_c.1090G>A_wt 55697 1000000
TP63_c.1090G>A_mt 237 1000000

In [47]:
for k, i in output1.items():
    print (k, len(i))

DNAJB1_c.558C>A_wt 1000
DNAJB1_c.558C>A_mt 622
FGFR1_c.719G>A_wt 1
FGFR1_c.719G>A_mt 69
KRAS_c.34G>T_wt 501
KRAS_c.34G>T_mt 69
PTPRC_c.2306T>A_wt 77
PTPRC_c.2306T>A_mt 4
TP63_c.1090G>A_wt 301
TP63_c.1090G>A_mt 84


In [36]:
x=pd.DataFrame.from_dict(target_dict, orient='index')

In [52]:
for k in target_dict.keys():
    s=k.split('_')[1][2:]+k.split('_')[2][0]
    s=s.replace('>','')
    if len(s)==7:
        print (s)
    else:
        print ((7-len(s))*'x'+s)

x558CAw
x558CAm
x719GAw
x719GAm
xx34GTw
xx34GTm
2306TAw
2306TAm
1090GAw
1090GAm


In [38]:
x.to_csv('targets.csv')

In [ ]:
# try on the rna fastqs

In [17]:
fastqgz= '/gstore/data/dnaseq/processed_runs/R9173/results/LIB5468563_HITS5485162_S7_L001_R2_001.fastq.gz'

In [ ]:

import gzip
from Bio import SeqIO
aligns={}
for k,i in target_dict.items():
    aligns[k]=[]
    
ontarget=0
parsedreads=0
with gzip.open(fastqgz, "rt") as handle:
    for record in SeqIO.parse(handle, "fastq"):
        parsedreads+=1
        if parsedreads<10000000:
            for k,i in target_dict.items():
                seq=record.seq
                tar=i
                if ontarget < 1000:
                    if str(tar) in str(seq):
                        aligns[k].append(record.id)
                        ontarget+=1   
                else:
                    continue
            

In [ ]:
for k, i in aligns.items():
    print (k, len(i))

DNAJB1_c.558C>A_wt 135
DNAJB1_c.558C>A_mt 125
FGFR1_c.719G>A_wt 0
FGFR1_c.719G>A_mt 7
KRAS_c.34G>T_wt 78
KRAS_c.34G>T_mt 2
PTPRC_c.2306T>A_wt 13
PTPRC_c.2306T>A_mt 2
TP63_c.1090G>A_wt 44
TP63_c.1090G>A_mt 11


In [ ]:
import pickle
with open('rna_aligns.pkl', 'wb') as file:
    pickle.dump(aligns, file)

In [ ]:
DNAJB1_c.558C>A_wt 1000
DNAJB1_c.558C>A_mt 622
FGFR1_c.719G>A_wt 1
FGFR1_c.719G>A_mt 69
KRAS_c.34G>T_wt 501
KRAS_c.34G>T_mt 69
PTPRC_c.2306T>A_wt 77
PTPRC_c.2306T>A_mt 4
TP63_c.1090G>A_wt 301
TP63_c.1090G>A_mt 84

## ddpadlock

In [7]:
ref=pd.read_csv('flex_dna_probes_230920.csv')

In [9]:
ref['target_LHS_mut']

0    TCCGTCGGGGTTTAGCCGCTTGTGT
1    ATTATGATGCTCCAGGTGGCATAAT
2    TCAAGGCACTCTTGCCTACGCCACA
3    CCAAAAGCCCGAGTGCCCTCTTCCT
4    CACCGTTCTTTGTACTGTCCGAAAT
Name: target_LHS_mut, dtype: object

0    TCCGTCGGGGTTTAGCCGCTTGTGT
1    ATTATGATGCTCCAGGTGGCATAAT
2    TCAAGGCACTCTTGCCTACGCCACA
3    CCAAAAGCCCGAGTGCCCTCTTCCT
4    CACCGTTCTTTGTACTGTCCGAAAT
Name: target_LHS_mut, dtype: object

In [15]:
ref['amplicon_ref']=ref['target_LHS_ref']+ref['target_RHS']
ref['amplicon_mut']=ref['target_LHS_mut']+ref['target_RHS']

# ref['cds_ref']=ref['target_LHS_mut']+ref['target_RHS']

In [49]:
# ll=[]
# for b in 'ATCG':
#     for c in 'ATCG':
#         ll.append(b+c)
        
# lll=list('ATCG')+ll
# lll

In [54]:
target_dict={}
for i,r in ref.iterrows():
    gene=r['Unnamed: 0']
    refs=r['amplicon_ref']
    muts=r['amplicon_mut']
    target_dict[gene+'_gap']=r['target_LHS_mut'][:-1]+r['target_RHS']
    # target_dict[gene+'_wt']=refs
    # target_dict[gene+'_mt']=muts
    for b in 'ATCG':
        target_dict[gene+'_'+b]=r['target_LHS_mut'][:-1]+b+r['target_RHS']
target_dict

{'DNAJB1_c.558C>A_A': 'TCCGTCGGGGTTTAGCCGCTTGTGAGAGATTTTCATCTTCTTGGTACAGC',
 'DNAJB1_c.558C>A_C': 'TCCGTCGGGGTTTAGCCGCTTGTGCGAGATTTTCATCTTCTTGGTACAGC',
 'DNAJB1_c.558C>A_G': 'TCCGTCGGGGTTTAGCCGCTTGTGGGAGATTTTCATCTTCTTGGTACAGC',
 'DNAJB1_c.558C>A_T': 'TCCGTCGGGGTTTAGCCGCTTGTGTGAGATTTTCATCTTCTTGGTACAGC',
 'DNAJB1_c.558C>A_gap': 'TCCGTCGGGGTTTAGCCGCTTGTGGAGATTTTCATCTTCTTGGTACAGC',
 'FGFR1_c.719G>A_A': 'ATTATGATGCTCCAGGTGGCATAAAGGACCTTGTAGCCTCCAATTCTGTG',
 'FGFR1_c.719G>A_C': 'ATTATGATGCTCCAGGTGGCATAACGGACCTTGTAGCCTCCAATTCTGTG',
 'FGFR1_c.719G>A_G': 'ATTATGATGCTCCAGGTGGCATAAGGGACCTTGTAGCCTCCAATTCTGTG',
 'FGFR1_c.719G>A_T': 'ATTATGATGCTCCAGGTGGCATAATGGACCTTGTAGCCTCCAATTCTGTG',
 'FGFR1_c.719G>A_gap': 'ATTATGATGCTCCAGGTGGCATAAGGACCTTGTAGCCTCCAATTCTGTG',
 'KRAS_c.34G>T_A': 'TCAAGGCACTCTTGCCTACGCCACAAGCTCCAACTACCACAAGTTTATAT',
 'KRAS_c.34G>T_C': 'TCAAGGCACTCTTGCCTACGCCACCAGCTCCAACTACCACAAGTTTATAT',
 'KRAS_c.34G>T_G': 'TCAAGGCACTCTTGCCTACGCCACGAGCTCCAACTACCACAAGTTTATAT',
 'KRAS_c.34G>T_T': 'TCAA

In [36]:
bulk_ddpadlock= '/gstore/data/ctgbioinfo/taol9/flex/ddpadlock/LIB5475883_SAM24438800_S2_L002_R2_001.fastq.gz'


In [60]:
ls -allh /gne/data/dnaseq/processed_runs/R9922/results/LIB6221273_SAM24439397*

-rw-r--r-- 1 seqlab dnaseq 192M Nov  1 01:27 /gne/data/dnaseq/processed_runs/R9922/results/LIB6221273_SAM24439397_S12_L001_I1_001.fastq.gz
-rw-r--r-- 1 seqlab dnaseq 209M Nov  1 01:27 /gne/data/dnaseq/processed_runs/R9922/results/LIB6221273_SAM24439397_S12_L001_I2_001.fastq.gz
-rw-r--r-- 2 seqlab dnaseq 509M Nov  1 01:27 /gne/data/dnaseq/processed_runs/R9922/results/LIB6221273_SAM24439397_S12_L001_R1_001.fastq.gz
-rw-r--r-- 2 seqlab dnaseq  133 Nov  1 04:13 /gne/data/dnaseq/processed_runs/R9922/results/LIB6221273_SAM24439397_S12_L001_R1_001.md5
-rw-r--r-- 2 seqlab dnaseq 722M Nov  1 01:27 /gne/data/dnaseq/processed_runs/R9922/results/LIB6221273_SAM24439397_S12_L001_R2_001.fastq.gz
-rw-r--r-- 2 seqlab dnaseq  133 Nov  1 04:13 /gne/data/dnaseq/processed_runs/R9922/results/LIB6221273_SAM24439397_S12_L001_R2_001.md5
-rw-r--r-- 1 seqlab dnaseq 184M Nov  1 01:58 /gne/data/dnaseq/processed_runs/R9922/results/LIB6221273_SAM24439397_S12_L002_I1_001.fastq.gz
-rw-r--r-- 1 seqlab dnaseq 197M Nov  

In [37]:
mut_only="/gne/data/dnaseq/processed_runs/R9922/results/LIB6221274_SAM24439398_S13_L002_R2_001.fastq.gz"
mix_mut='/gne/data/dnaseq/processed_runs/R9922/results/LIB6221273_SAM24439397_S12_L002_R2_001.fastq.gz'

In [ ]:
#zcat foo.gz | bgzip -c > foo.bgz

In [56]:
# target_dict={'DNAJB1_c.558C>A_mt': 'TCCGTCGGGGTTTAGCCGCTTGTGTGAGATTTTCATCTTCTTGGTACAGC',
#  'DNAJB1_c.558C>A_wt': 'TCCGTCGGGGTTTAGCCGCTTGTGGGAGATTTTCATCTTCTTGGTACAGC',
#  'FGFR1_c.719G>A_mt': 'ATTATGATGCTCCAGGTGGCATAATGGACCTTGTAGCCTCCAATTCTGTG',
#  'FGFR1_c.719G>A_wt': 'ATTATGATGCTCCAGGTGGCATAACGGACCTTGTAGCCTCCAATTCTGTG',
#  'KRAS_c.34G>T_mt': 'TCAAGGCACTCTTGCCTACGCCACAAGCTCCAACTACCACAAGTTTATAT',
#  'KRAS_c.34G>T_wt': 'TCAAGGCACTCTTGCCTACGCCACCAGCTCCAACTACCACAAGTTTATAT',
#  'PTPRC_c.2306T>A_mt': 'CCAAAAGCCCGAGTGCCCTCTTCCTTTGACGGCCAGTATTCTGCACACTT',
#  'PTPRC_c.2306T>A_wt': 'CCAAAAGCCCGAGTGCCCTCTTCCATTGACGGCCAGTATTCTGCACACTT',
#  'TP63_c.1090G>A_mt': 'CACCGTTCTTTGTACTGTCCGAAATTTGCTGCTTTCTGATGCTATCTTCA',
#  'TP63_c.1090G>A_wt': 'CACCGTTCTTTGTACTGTCCGAAACTTGCTGCTTTCTGATGCTATCTTCA'}

def bulk_mut_counts(fastqgz,target_dict=target_dict,maxreads=1000000):
    aligns={}
    for k,i in target_dict.items():
        aligns[k]=[]
    
    ontarget=0
    parsedreads=0
    with gzip.open(fastqgz, "rt") as handle:
        for record in SeqIO.parse(handle, "fastq"):
            parsedreads+=1
            if parsedreads<=maxreads:
                for k,i in target_dict.items():
                    seq=record.seq
                    tar=i
                    # if ontarget < 1000:
                    if str(tar) in str(seq):
                        aligns[k].append(record.id)
                            # aligns[k].append(record.seq)

                        ontarget+=1   
                    # else:
                    #     continue
    print ('mapping rate: ',ontarget/parsedreads)

    for k, i in aligns.items():
        print (k, len(i))

In [57]:
bulk_mut_counts(bulk_ddpadlock,maxreads=1000000)

mapping rate:  5.184266206274591e-05
DNAJB1_c.558C>A_gap 300
DNAJB1_c.558C>A_A 0
DNAJB1_c.558C>A_T 0
DNAJB1_c.558C>A_C 0
DNAJB1_c.558C>A_G 12
FGFR1_c.719G>A_gap 0
FGFR1_c.719G>A_A 0
FGFR1_c.719G>A_T 0
FGFR1_c.719G>A_C 0
FGFR1_c.719G>A_G 0
KRAS_c.34G>T_gap 0
KRAS_c.34G>T_A 9
KRAS_c.34G>T_T 0
KRAS_c.34G>T_C 8
KRAS_c.34G>T_G 0
PTPRC_c.2306T>A_gap 2
PTPRC_c.2306T>A_A 0
PTPRC_c.2306T>A_T 0
PTPRC_c.2306T>A_C 0
PTPRC_c.2306T>A_G 0
TP63_c.1090G>A_gap 0
TP63_c.1090G>A_A 0
TP63_c.1090G>A_T 0
TP63_c.1090G>A_C 0
TP63_c.1090G>A_G 0


In [58]:
bulk_mut_counts(mix_mut,maxreads=1000000)

mapping rate:  0.012536560689524807
DNAJB1_c.558C>A_gap 19248
DNAJB1_c.558C>A_A 16
DNAJB1_c.558C>A_T 684
DNAJB1_c.558C>A_C 0
DNAJB1_c.558C>A_G 25854
FGFR1_c.719G>A_gap 0
FGFR1_c.719G>A_A 0
FGFR1_c.719G>A_T 752
FGFR1_c.719G>A_C 0
FGFR1_c.719G>A_G 0
KRAS_c.34G>T_gap 11419
KRAS_c.34G>T_A 194135
KRAS_c.34G>T_T 5
KRAS_c.34G>T_C 17510
KRAS_c.34G>T_G 103
PTPRC_c.2306T>A_gap 4326
PTPRC_c.2306T>A_A 0
PTPRC_c.2306T>A_T 0
PTPRC_c.2306T>A_C 0
PTPRC_c.2306T>A_G 0
TP63_c.1090G>A_gap 27
TP63_c.1090G>A_A 1
TP63_c.1090G>A_T 13675
TP63_c.1090G>A_C 30329
TP63_c.1090G>A_G 519


In [59]:
bulk_mut_counts(mut_only,maxreads=1000000)

mapping rate:  0.0023743818900329003
DNAJB1_c.558C>A_gap 7963
DNAJB1_c.558C>A_A 0
DNAJB1_c.558C>A_T 638
DNAJB1_c.558C>A_C 24
DNAJB1_c.558C>A_G 7773
FGFR1_c.719G>A_gap 0
FGFR1_c.719G>A_A 0
FGFR1_c.719G>A_T 37
FGFR1_c.719G>A_C 3
FGFR1_c.719G>A_G 0
KRAS_c.34G>T_gap 12634
KRAS_c.34G>T_A 30440
KRAS_c.34G>T_T 18
KRAS_c.34G>T_C 3864
KRAS_c.34G>T_G 27
PTPRC_c.2306T>A_gap 4233
PTPRC_c.2306T>A_A 9
PTPRC_c.2306T>A_T 0
PTPRC_c.2306T>A_C 0
PTPRC_c.2306T>A_G 0
TP63_c.1090G>A_gap 136
TP63_c.1090G>A_A 0
TP63_c.1090G>A_T 1857
TP63_c.1090G>A_C 3361
TP63_c.1090G>A_G 170


# single cell

In [22]:
import gzip
from Bio import SeqIO
target_dict={'DNAJB1_c.558C>A_mt': 'TCCGTCGGGGTTTAGCCGCTTGTGTGAGATTTTCATCTTCTTGGTACAGC',
 'DNAJB1_c.558C>A_wt': 'TCCGTCGGGGTTTAGCCGCTTGTGGGAGATTTTCATCTTCTTGGTACAGC',
 'FGFR1_c.719G>A_mt': 'ATTATGATGCTCCAGGTGGCATAATGGACCTTGTAGCCTCCAATTCTGTG',
 'FGFR1_c.719G>A_wt': 'ATTATGATGCTCCAGGTGGCATAACGGACCTTGTAGCCTCCAATTCTGTG',
 'KRAS_c.34G>T_mt': 'TCAAGGCACTCTTGCCTACGCCACAAGCTCCAACTACCACAAGTTTATAT',
 'KRAS_c.34G>T_wt': 'TCAAGGCACTCTTGCCTACGCCACCAGCTCCAACTACCACAAGTTTATAT',
 'PTPRC_c.2306T>A_mt': 'CCAAAAGCCCGAGTGCCCTCTTCCTTTGACGGCCAGTATTCTGCACACTT',
 'PTPRC_c.2306T>A_wt': 'CCAAAAGCCCGAGTGCCCTCTTCCATTGACGGCCAGTATTCTGCACACTT',
 'TP63_c.1090G>A_mt': 'CACCGTTCTTTGTACTGTCCGAAATTTGCTGCTTTCTGATGCTATCTTCA',
 'TP63_c.1090G>A_wt': 'CACCGTTCTTTGTACTGTCCGAAACTTGCTGCTTTCTGATGCTATCTTCA'}

def flex_mutation_mapper(fastqR1, fastqR2,target_dict,pickle_name,whitelist="/gstore/home/taol9/gitsss/yo/flex/737K-fixed-rna-profiling.txt"):
    from collections import defaultdict
    import pandas as pd
    import pickle
    wl=[]
    f=open(whitelist,"r")
    wl=[l.strip() for l in f.readlines()]
    f.close()
    barcodes_dict = SeqIO.index(fastqR1,'fastq')
    record_dict = SeqIO.index(fastqR2,'fastq')
    mutation_matrix=defaultdict(lambda: defaultdict(list))

    for i in list(record_dict.keys()):
        record=record_dict[i]
        seq=record.seq
        try:
            barcode_record=barcodes_dict[i]
            barcode_seq=barcode_record.seq
            barcode=str(barcode_seq[:16])
            umi=str(barcode_seq[16:])
    
            for tar_k, tar in target_dict.items():
                if str(tar) in str(seq) and barcode in wl:
                    mutation_matrix[barcode][tar_k].append(umi)   
                else:
                    continue
        except:
            continue
            
    to_df=dict()
    for k,i in mutation_matrix.items():
        tmp={}
        for kk,ii in i.items():
            tmp[kk]=len(Counter(ii))
        to_df[k]=tmp
    df=pd.DataFrame.from_dict(to_df, orient='index')
    df.sort_index(axis = 1, inplace=True)

    with open(pickle_name, 'wb') as file:
        pickle.dump(df, file)
        
    return df, mutation_matrix

In [3]:
# cp  /gne/data/dnaseq/processed_runs/R9922/results/LIB6221274_SAM24439398*  /gstore/home/taol9/ctgbioinfo/taol9/flex/ddpadlock/R9922/

In [8]:
# zcat *.gz | bgzip -c > foo.bgz

In [ ]:
ls /gstore/home/taol9/ctgbioinfo/taol9/flex/ddpadlock/R9922/

In [15]:
fastqR1="/gstore/home/taol9/ctgbioinfo/taol9/flex/ddpadlock/R9922/LIB6221274_SAM24439398_S13_L001_R1_001.fastq.bgz"
fastqR2="/gstore/home/taol9/ctgbioinfo/taol9/flex/ddpadlock/R9922/LIB6221274_SAM24439398_S13_L001_R2_001.fastq.bgz"


In [ ]:
ddpadlock_df, ddpadlock_mm=flex_mutation_mapper(fastqR1, fastqR2,target_dict=target_dict,pickle_name='mut_only_ddpadlock_mix_mut.pickle',whitelist="/gstore/home/taol9/gitsss/yo/flex/737K-fixed-rna-profiling.txt")

In [17]:
## filter by cell barcodes from cell ranger

In [62]:
def rc(x):
    return str(Seq(x).reverse_complement())

In [74]:
fil_bc='/gstore/scratch/u/lauj22/NGS5110_ddPadlock/ddpadlock_only_Mutation/LIB6221274_SAM24439398/outs/per_sample_outs/LIB6221274_SAM24439398/count/sample_filtered_barcodes.csv'

cbcr=[x[:-2] for x in pd.read_csv(fil_bc, header=None)[1]]
to_df={}
for k,i in ddpadlock_mm.items():
    tmp={}
    if k in cbcr:
        for kk,ii in i.items():
            tmp[kk]=len(Counter(ii))
        to_df[k]=tmp

In [78]:
df=pd.DataFrame.from_dict(to_df,orient='index')

In [27]:
ddpadlock_df['DNAJB1_c.558C>A_mt'].value_counts()

1.0     44
2.0      6
4.0      4
5.0      3
3.0      2
7.0      1
11.0     1
12.0     1
10.0     1
Name: DNAJB1_c.558C>A_mt, dtype: int64

In [34]:
condition=((ddpadlock_df['DNAJB1_c.558C>A_wt']>0)
           &(ddpadlock_df['DNAJB1_c.558C>A_mt']>0)
# &(df['PTPRC_c.2306T>A_wt']>0)
# &(df['KRAS_c.34G>T_wt']>0)
# &(df['KRAS_c.34G>T_mt']>0)
# &(df['TP63_c.1090G>A_wt']>0)
# &(df['FGFR1_c.719G>A_wt']>0)
          )

In [81]:
def loci(x):
    l=0
    for i in x:
        if i>0:
            l+=1
    return l
    

In [82]:
ddpadlock_df['loci']=ddpadlock_df.apply(loci, axis=1)

In [83]:
ddpadlock_df['loci'].value_counts()

1    2977
2     274
3      13
Name: loci, dtype: int64

In [40]:
kras=((ddpadlock_df['KRAS_c.34G>T_wt']>0)&(ddpadlock_df['KRAS_c.34G>T_mt']>0))
ddpadlock_df[kras]

,DNAJB1_c.558C>A_mt,DNAJB1_c.558C>A_wt,FGFR1_c.719G>A_mt,FGFR1_c.719G>A_wt,KRAS_c.34G>T_mt,KRAS_c.34G>T_wt,PTPRC_c.2306T>A_wt,TP63_c.1090G>A_mt,TP63_c.1090G>A_wt
AAACGTTCAAGTCGCG,NaN,NaN,NaN,NaN,7.0,1.0,NaN,NaN,NaN
AAACGTTCAAGTTGGC,NaN,NaN,NaN,NaN,9.0,1.0,NaN,NaN,NaN
AAAGGCTTCATTAGTC,NaN,NaN,NaN,NaN,5.0,1.0,NaN,NaN,NaN
AAAGTCATCAGGCTGA,NaN,NaN,NaN,NaN,3.0,1.0,NaN,NaN,NaN
AACATAGTCACTATGT,NaN,NaN,NaN,NaN,3.0,1.0,NaN,NaN,NaN
AACCTGCTCTGCCATA,NaN,NaN,NaN,NaN,3.0,2.0,NaN,NaN,NaN
AACCTTTAGCGCTACT,NaN,NaN,NaN,NaN,4.0,1.0,NaN,NaN,NaN
AAGACAACAAGCTATG,NaN,NaN,NaN,NaN,8.0,1.0,NaN,NaN,NaN
AAGGAGCAGTAACGTA,NaN,NaN,NaN,NaN,7.0,1.0,NaN,NaN,NaN
AATCACGCATTTCGCG,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN
